In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer,OneHotEncoder,LabelEncoder,StandardScaler
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score,classification_report
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTENC



import os

In [3]:
df=pd.read_csv('DoctorFeePrediction_Milestone2.csv')

In [4]:
df.head()



,Doctor Name,City,Specialization,Doctor Qualification,Experience(Years),Total_Reviews,Patient Satisfaction Rate(%age),Avg Time to Patients(mins),Wait Time(mins),Hospital Address,Doctors Link,Fee Category
0,Dr. Umair Hafeez,GUJRANWALA,Ent Specialist,"MBBS , FCPS ( Otorhinolaryngologic ENT )",6.0,11,100,19,6,"Central Hospital, Jinnah Colony, Gujranwala",https://www.marham.pk/doctors/gujranwala/ent-s...,Expensive
1,Dr. Haris Shakeel,RAJAN-PUR,General Physician,MBBS,1.0,0,94,14,11,No Address Available,No Link Available,Cheap
2,Dr. Iqra Rehman,MIRPUR-KHAS,General Physician,"MBBS, RMP, CFP (USA), Certified in Covid 19 +",6.0,9,100,10,0,"Rehman Clinic, tandoadam naka, Mirpur Khas",https://www.marham.pk/doctors/mirpur-khas/gene...,Medium-Priced
3,Dr. Erum Memon,HYDERABAD,Gynecologist,"MBBS, FCPS (Gynae &amp; Obstetrics)",11.0,71,96,18,10,"Mehmood Hospital, Qasimabad, Hyderabad",https://www.marham.pk/doctors/hyderabad/gyneco...,Cheap
4,Dr. Aisha Ahmad,LAHORE,Dermatologist,"MBBS, FCPS (Dermatology), Certified (Aesthetic...",12.0,199,100,16,2,"Skinnovation, Johar Town, Lahore",https://www.marham.pk/doctors/lahore/dermatolo...,Medium-Priced


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2386 entries, 0 to 2385
Data columns (total 12 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Doctor Name                      2386 non-null   object 
 1   City                             2386 non-null   object 
 2   Specialization                   2386 non-null   object 
 3   Doctor Qualification             2386 non-null   object 
 4   Experience(Years)                2386 non-null   float64
 5   Total_Reviews                    2386 non-null   int64  
 6   Patient Satisfaction Rate(%age)  2386 non-null   int64  
 7   Avg Time to Patients(mins)       2386 non-null   int64  
 8   Wait Time(mins)                  2386 non-null   int64  
 9   Hospital Address                 2386 non-null   object 
 10  Doctors Link                     2386 non-null   object 
 11  Fee Category                     2386 non-null   object 
dtypes: float64(1), int64

In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Experience(Years),2386.0,11.844719,8.784449,1.0,6.0,10.0,14.0,53.0
Total_Reviews,2386.0,92.473177,282.162526,0.0,0.0,8.0,54.0,5147.0
Patient Satisfaction Rate(%age),2386.0,96.657586,4.962300,33.0,94.0,98.0,100.0,100.0
Avg Time to Patients(mins),2386.0,14.092205,2.722198,3.0,14.0,14.0,15.0,50.0
Wait Time(mins),2386.0,11.264459,5.636885,0.0,10.0,11.0,11.0,82.0


In [7]:
df['Fee Category'].value_counts()

Medium-Priced    1042
Cheap             864
Expensive         480
Name: Fee Category, dtype: int64

In [8]:
df.Specialization=df.Specialization.str.replace(" ",'').str.lower().str.strip()

In [9]:
df.City=df.City.str.replace(" ",'').str.lower().str.strip()

In [10]:
df["Doctor Qualification"]=df["Doctor Qualification"].str.lower().str.replace("*","").str.replace(']','}').str.replace('[','{').str.replace('(','{').str.replace(')',"}").str.strip().str.split(",")

C:\Users\lenovo\AppData\Local\Temp\ipykernel_64208\3702064878.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["Doctor Qualification"]=df["Doctor Qualification"].str.lower().str.replace("*","").str.replace(']','}').str.replace('[','{').str.replace('(','{').str.replace(')',"}").str.strip().str.split(",")


In [11]:
df['Doctor Qualification']=df['Doctor Qualification'].apply(lambda x:','.join(x))
df['Doctor Qualification']=df['Doctor Qualification'].str.replace(" ",'')

## Getting Doctor title

In [12]:
df["Title"]=df["Doctor Name"].str.strip().str.split(".").apply(lambda x : x[0]).str.capitalize()
df.Title.value_counts()

Dr                      1908
Asst                     249
Prof                     149
Assoc                     79
Mohammad zafar iqbal       1
Name: Title, dtype: int64

In [13]:
## Later for encoding
doctors_rank={
    "Prof":3,
    "Assoc":2,
    "Asst":1,
    "Dr":0
}

df["Title"]=df["Title"].map(doctors_rank)
df['Title'].value_counts(dropna=False)


0.0    1908
1.0     249
3.0     149
2.0      79
NaN       1
Name: Title, dtype: int64

In [14]:
df.Title.fillna(0,inplace=True)
df['Title'].value_counts(dropna=False)

0.0    1909
1.0     249
3.0     149
2.0      79
Name: Title, dtype: int64

## Fee Category

In [15]:
df['Fee Category'].value_counts(dropna=False)

Medium-Priced    1042
Cheap             864
Expensive         480
Name: Fee Category, dtype: int64

In [16]:
doctors_fee={
    'Cheap':0,
    'Medium-Priced':1,
    'Expensive':2
}
df['Fee Category']=df['Fee Category'].map(doctors_fee)
df['Fee Category'].value_counts(dropna=False)


1    1042
0     864
2     480
Name: Fee Category, dtype: int64

In [17]:
df.head()

,Doctor Name,City,Specialization,Doctor Qualification,Experience(Years),Total_Reviews,Patient Satisfaction Rate(%age),Avg Time to Patients(mins),Wait Time(mins),Hospital Address,Doctors Link,Fee Category,Title
0,Dr. Umair Hafeez,gujranwala,entspecialist,"mbbs,fcps{otorhinolaryngologicent}",6.0,11,100,19,6,"Central Hospital, Jinnah Colony, Gujranwala",https://www.marham.pk/doctors/gujranwala/ent-s...,2,0.0
1,Dr. Haris Shakeel,rajan-pur,generalphysician,mbbs,1.0,0,94,14,11,No Address Available,No Link Available,0,0.0
2,Dr. Iqra Rehman,mirpur-khas,generalphysician,"mbbs,rmp,cfp{usa},certifiedincovid19+",6.0,9,100,10,0,"Rehman Clinic, tandoadam naka, Mirpur Khas",https://www.marham.pk/doctors/mirpur-khas/gene...,1,0.0
3,Dr. Erum Memon,hyderabad,gynecologist,"mbbs,fcps{gynae&amp;obstetrics}",11.0,71,96,18,10,"Mehmood Hospital, Qasimabad, Hyderabad",https://www.marham.pk/doctors/hyderabad/gyneco...,0,0.0
4,Dr. Aisha Ahmad,lahore,dermatologist,"mbbs,fcps{dermatology},certified{aestheticmedi...",12.0,199,100,16,2,"Skinnovation, Johar Town, Lahore",https://www.marham.pk/doctors/lahore/dermatolo...,1,0.0


In [18]:
X_train,X_test,y_train,y_test=train_test_split(df.drop(columns=['Fee Category','Hospital Address','Doctor Name','Doctors Link']),df['Fee Category'],random_state=42,test_size=0.2,stratify=df['Fee Category'])

# Train

In [19]:
def log_transform(X, columns_to_transform):
    X_transformed = X.copy()
    for col in columns_to_transform:
        X_transformed[col] = np.log1p(X_transformed[col])
    return X_transformed

In [20]:
log_transformer=FunctionTransformer(func=log_transform, kw_args={'columns_to_transform': ['Experience(Years)', 'Total_Reviews']})

In [21]:
X_train=log_transformer.fit_transform(X_train)

In [22]:
enc = pd.get_dummies(X_train['City'], prefix='City')
X_train=pd.concat([X_train.drop(columns='City'),enc],axis=1)
enc.shape

(1908, 113)

In [23]:
enc=X_train.Specialization.str.get_dummies(sep=',')
X_train=pd.concat([X_train.drop(columns='Specialization'),enc],axis=1)
enc.shape

(1908, 60)

In [24]:
X_train

,Doctor Qualification,Experience(Years),Total_Reviews,Patient Satisfaction Rate(%age),Avg Time to Patients(mins),Wait Time(mins),Title,City_abbottabad,City_alipur,City_attock,...,psychiatrist,psychologist,pulmonologist/lungspecialist,radiologist,rehabilitationmedicine,rheumatologist,sexologist,spinalsurgeon,spinespecialist,urologist
596,"mbbs,mcps,{{ophthalmology},fcps{ophthalmology}",3.367296,1.609438,100,14,11,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2097,"mbbs,mrcp,mrcps,dcn",2.397895,0.000000,94,14,11,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
334,mbbs,1.609438,1.098612,100,14,11,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
727,"mbbs,fcps",2.197225,0.000000,94,14,11,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1040,"mbbs,mcps{obstetrics&amp;gynecology}",2.079442,0.000000,94,14,11,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,"mbbs,fcps{urology}",2.397895,5.030438,99,14,9,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
478,"mbbs,fcps{paeds},mrcp{ireland},facp{usa}",2.564949,4.770685,99,17,9,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
271,"mbbs,fcps{gastroenterology},mrcp1,fcps1{medicine}",2.708050,5.303305,98,16,11,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
967,"mbbs,fcps{obstetrics&amp;gynecology}",2.079442,0.000000,94,14,11,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
enc=X_train['Doctor Qualification'].str.get_dummies(sep=',')
X_train=pd.concat([X_train.drop(columns=['Doctor Qualification']),enc],axis=1)
enc.shape

(1908, 865)

In [26]:
X_train

,Experience(Years),Total_Reviews,Patient Satisfaction Rate(%age),Avg Time to Patients(mins),Wait Time(mins),Title,City_abbottabad,City_alipur,City_attock,City_baden,...,{dermatology},{fellowshiptrainingincutaneous&amp;lasersurgery{thailand},{gynecology},{karachi}mcps,{london},{neurosurgery},{neurosurgery}i.t.f{unitedkingdom},{orthopedicsurgery,{{ophthalmology},fcps{ophthalmology}
596,3.367296,1.609438,100,14,11,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2097,2.397895,0.000000,94,14,11,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
334,1.609438,1.098612,100,14,11,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
727,2.197225,0.000000,94,14,11,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1040,2.079442,0.000000,94,14,11,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,2.397895,5.030438,99,14,9,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
478,2.564949,4.770685,99,17,9,1.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
271,2.708050,5.303305,98,16,11,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
967,2.079442,0.000000,94,14,11,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Test

In [27]:
X_test=log_transformer.transform(X_test)

In [28]:
enc = pd.get_dummies(X_test['City'], prefix='City')
enc1=X_test.Specialization.str.get_dummies(sep=',')
enc2=X_test['Doctor Qualification'].str.get_dummies(sep=',')
X_test=pd.concat([X_test.drop(columns=['City','Specialization','Doctor Qualification']),enc,enc1,enc2],axis=1)
enc.shape

(478, 88)

In [29]:
test = pd.DataFrame(columns=X_train.columns)

In [30]:
test

,Experience(Years),Total_Reviews,Patient Satisfaction Rate(%age),Avg Time to Patients(mins),Wait Time(mins),Title,City_abbottabad,City_alipur,City_attock,City_baden,...,{dermatology},{fellowshiptrainingincutaneous&amp;lasersurgery{thailand},{gynecology},{karachi}mcps,{london},{neurosurgery},{neurosurgery}i.t.f{unitedkingdom},{orthopedicsurgery,{{ophthalmology},fcps{ophthalmology}


In [31]:
test = pd.DataFrame(columns=X_train.columns)
for col in test.columns:
        if col in X_test.columns:
            test[col] = X_test[col]
        else:
            test[col] = 0

In [32]:
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier

model=RandomForestClassifier(random_state=42)
logistic_reg = LogisticRegression(multi_class='multinomial',solver='lbfgs')
svm = SVC(kernel='linear')  # Linear kernel for SVM
decision_tree = DecisionTreeClassifier(criterion='entropy', random_state=42)
AdaBoostClass = AdaBoostClassifier(random_state=42)
CatBoss = CatBoostClassifier(random_state=42)

# pipeline = Pipeline(steps=[('preprocessor', preprocessor),
#                            ('model', model)])
model.fit(X_train, y_train)
print("Train: ",model.score(X_train, y_train))
print("Test: ",model.score(test,y_test))
logistic_reg.fit(X_train, y_train)
print("Train: ",logistic_reg.score(X_train, y_train))
print("Test: ",logistic_reg.score(test,y_test))
svm.fit(X_train, y_train)
print("Train: ",svm.score(X_train, y_train))
print("Test: ",svm.score(test,y_test))
decision_tree.fit(X_train, y_train)
print("Train: ",decision_tree.score(X_train, y_train))
print("Test: ",decision_tree.score(test,y_test))
AdaBoostClass.fit(X_train, y_train)
print("Train: ",AdaBoostClass.score(X_train, y_train))
print("Test: ",AdaBoostClass.score(test,y_test))

Train:  0.9947589098532494
Test:  0.6673640167364017


c:\Users\lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Train:  0.6257861635220126
Test:  0.5460251046025104
Train:  0.8160377358490566
Test:  0.6317991631799164
Train:  0.9947589098532494
Test:  0.5774058577405857
Train:  0.6388888888888888
Test:  0.5585774058577406
